Project-27

In [2]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [ ]:
Name : Muze Zhao

In [6]:

covid_vaccine <- read_csv("https://raw.githubusercontent.com/Carson-Lu/stat201project-Group27/main/covid-vaccination-vs-death_ratio.csv")
covid_vaccine

New names:
• `` -> `...1`
Rows: 32911 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): country, iso_code
dbl  (7): ...1, total_vaccinations, people_vaccinated, people_fully_vaccinat...
date (1): date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


...1,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,New_deaths,population,ratio
<dbl>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,Afghanistan,AFG,2021-05-11,504502,448878,55624,12,40374668,1.111781
1,Afghanistan,AFG,2021-05-20,547901,470341,77560,10,40374668,1.164941
2,Afghanistan,AFG,2021-05-24,573277,476367,96910,10,40374668,1.179866
3,Afghanistan,AFG,2021-05-26,590454,479372,111082,19,40374668,1.187309
4,Afghanistan,AFG,2021-05-27,593313,479574,113739,14,40374668,1.187809
5,Afghanistan,AFG,2021-05-30,600152,480226,119926,20,40374668,1.189424
6,Afghanistan,AFG,2021-06-02,626290,481690,144600,34,40374668,1.193050
7,Afghanistan,AFG,2021-06-03,630305,481800,148505,27,40374668,1.193323
8,Afghanistan,AFG,2021-06-08,641295,482952,158343,64,40374668,1.196176


In [14]:
covid_vaccine |> 
filter(country == "Canada") 

...1,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,New_deaths,population,ratio
<dbl>,<chr>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
5235,Canada,CAN,2020-12-22,26862,26861,1,79,38261228,0.07020423
5236,Canada,CAN,2020-12-23,35547,35546,1,67,38261228,0.09290345
5237,Canada,CAN,2020-12-24,45745,45744,1,93,38261228,0.11955706
5238,Canada,CAN,2020-12-25,48792,48791,1,169,38261228,0.12752074
5239,Canada,CAN,2020-12-26,48808,48807,1,152,38261228,0.12756255
5240,Canada,CAN,2020-12-27,52419,52418,1,85,38261228,0.13700031
5241,Canada,CAN,2020-12-28,59041,59040,1,167,38261228,0.15430765
5242,Canada,CAN,2020-12-29,72057,72055,2,104,38261228,0.18832380
5243,Canada,CAN,2020-12-30,85497,85495,2,113,38261228,0.22345075
